In [1]:
import base64
from io import BytesIO
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.ticker import FuncFormatter
import seaborn as sns
import yfinance as yf
from backend.volatility.estimators import VolatilityEstimator, multi_window_estimates

In [2]:
estimators = [
    "close_to_close",
    "parkinson",
    "garman_klass",
    "rogers_satchell",
    "yang_zhang",
]

windows = [30, 60, 90, 120]

spx = yf.Ticker("^SPX")
quotes = spx.history(period="1y")
ens = VolatilityEstimator(estimators=estimators)
vols = multi_window_estimates(
    estimator=ens,
    price_data=quotes,
    windows=windows,
    components=True)

In [3]:
vols

Estimator,close_to_close,parkinson,garman_klass,rogers_satchell,yang_zhang,mean,close_to_close,parkinson,garman_klass,rogers_satchell,...,garman_klass,rogers_satchell,yang_zhang,mean,close_to_close,parkinson,garman_klass,rogers_satchell,yang_zhang,mean
Window,30,30,30,30,30,30,60,60,60,60,...,90,90,90,90,120,120,120,120,120,120
Date,,,,,,,,,,,,,,,,,,,,,
2023-06-13,0.124716,0.094909,0.093891,0.094195,0.113435,0.104229,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-06-14,0.118296,0.092124,0.091986,0.091782,0.110903,0.101018,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-06-15,0.118626,0.093668,0.091843,0.090298,0.110448,0.100976,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-06-16,0.116479,0.093903,0.091773,0.089905,0.110245,0.100461,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-06-20,0.108610,0.091077,0.090544,0.089693,0.107372,0.097459,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-04-23,0.118277,0.100941,0.098157,0.095990,0.118630,0.106399,0.122282,0.090351,0.088203,0.087048,...,0.085156,0.084495,0.106625,0.095699,0.112897,0.084550,0.082046,0.080899,0.103667,0.092812
2024-04-24,0.113217,0.099563,0.097133,0.095374,0.117460,0.104549,0.121428,0.090329,0.088574,0.087722,...,0.085106,0.084880,0.106640,0.095220,0.112695,0.084402,0.081943,0.080827,0.103626,0.092699


In [4]:
df_long = vols.stack(level=[0, 1], future_stack=True).reset_index()
df_long.columns = ['Index', 'Estimator', 'Window', 'Value']
df_long

,Index,Estimator,Window,Value
0,2023-06-13,close_to_close,30,0.124716
1,2023-06-13,parkinson,30,0.094909
2,2023-06-13,garman_klass,30,0.093891
3,2023-06-13,rogers_satchell,30,0.094195
4,2023-06-13,yang_zhang,30,0.113435
...,...,...,...,...
5299,2024-04-29,parkinson,120,0.084121
5300,2024-04-29,garman_klass,120,0.082518
5301,2024-04-29,rogers_satchell,120,0.081801
5302,2024-04-29,yang_zhang,120,0.104648


In [ ]:
# Plotting
sns.boxplot(x='Window', y='Value', hue='Estimator', data=df_long, palette='muted')
plt.title('Multi-Index Boxplot')
plt.show()

In [ ]:
# Draw a nested violinplot and split the violins for easier comparison
sns.violinplot(data=df_long, x="Window", y="Value", hue="Estimator",
               split=True, inner="quart", fill=False,
               palette='muted')

In [ ]:
sns.violinplot(data=df_long, bw_adjust=.5, cut=1, linewidth=1, palette="Set3")

In [ ]:
def to_percentage(value, _):
    return f'{100 * value:.1f}%'

# Figure setup
plt.figure(figsize=(9,5), dpi=600)
left, width = 0.09, 0.65
bottom, height = 0.1, 0.85
left_h = left+width+0.02
rect_left = [left, bottom, width, height]
rect_right = [left_h, bottom, 0.17, height]
left = plt.axes(rect_left)
right = plt.axes(rect_right)

# First subplot using Seaborn
sns.lineplot(
    data,
    palette="vlag_r",
    errorbar="sd",
    dashes=False,
    ax=left)

# Setting date-specific x-ticks every 20 days
end_date = pd.Timestamp('today')
start_date = end_date - pd.Timedelta(days=60)
left.set_xlim(start_date, end_date)

# Set x-axis major ticks to every 15 days
left.xaxis.set_major_locator(mdates.DayLocator(interval=15))
left.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
left.legend(fontsize='small')

# Rotate date labels to prevent overlap
plt.gcf().autofmt_xdate()

# Second subplot
sns.boxplot(
    data,
    notch=True,
    palette="vlag_r",
    fliersize=7,
    flierprops={'marker':'x'},
    ax=right)
sns.stripplot(data, size=2, color=".3", ax=right)
sns.lineplot(data.mean(axis=0), color="blue", zorder=5, label="mean", ax=right)
sns.lineplot(data.iloc[-1], color="red", ax=right, label="current")

# change x-ticks
current_labels = [item.get_text() for item in right.get_xticklabels()]
right.set_xticks(range(len(current_labels)))
right.set_xticklabels(range(1, len(current_labels) + 1))
right.legend(fontsize='small')

# change y-ticks
left.yaxis.set_major_formatter(FuncFormatter(to_percentage))
right.yaxis.set_major_formatter(FuncFormatter(to_percentage))
right.yaxis.tick_right()
right.set_ylabel("")

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Creating a multi-index DataFrame
arrays = [['A', 'A', 'B', 'B'], ['one', 'two', 'one', 'two']]
index = pd.MultiIndex.from_arrays(arrays, names=('Top Level', 'Sub Level'))
df = pd.DataFrame(np.random.randn(5, 4), columns=index)

print("Original DataFrame:")
print(df)

In [ ]:
# Melt the DataFrame
df_long = df.stack(level=[0, 1], future_stack=True).reset_index()
df_long.columns = ['Index', 'Top Level', 'Sub Level', 'Value']

print("\nTransformed DataFrame:")
print(df_long)


In [ ]:
# Plotting
sns.boxplot(x='Top Level', y='Value', hue='Sub Level', data=df_long)
plt.title('Multi-Index Boxplot')
plt.show()